In [1]:
%%capture
!pip install git+https://github.com/huggingface/transformers.git -q -U # transformers version:  4.37.0
!pip install git+https://github.com/huggingface/accelerate.git -q -U # accelerate version:  0.27.0
!pip install bitsandbytes # bitsandbytes version:  0.42.0
!pip install git+https://github.com/huggingface/peft.git -q -U # peft version: 0.7.2
# pip install bitsandbytes

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
!pip install torch

In [4]:
pip install --upgrade bitsandbytes

Note: you may need to restart the kernel to use updated packages.


In [5]:
model_3 = AutoModelForCausalLM.from_pretrained(
        'ashishkgpian/full_v5_astromistral_final',
        torch_dtype=torch.bfloat16,
        device_map="auto",
        trust_remote_code=True,)
tokenizer = AutoTokenizer.from_pretrained('mistralai/Mistral-7B-Instruct-v0.1', trust_remote_code=True)

config.json:   0%|          | 0.00/1.04k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.45G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [6]:
abstract = 'Diabetes is now one of the major public health challenges, globally. Prolonged diabetes leads to various diabetic microvascular complications (DMCs) like retinopathy, nephropathy, and neuropathy. Multiple factors are likely to be involved in predisposing diabetic individuals to complications. Early detection or diagnosis is essential in developing strategies to reduce the risk factors and management costs of these diabetic complications. In this study, we employed Raman Spectroscopy (RS) to analyse the plasma samples of diabetes patients without and with DMCs along with the plasma samples of healthy subjects. Spectral comparisons revealed decrease in protein content in Diabetes group and further subsequent decrease in proteins in DMC groups when compared with control group, which corroborates with the fact that there exists increased secretion of proteins in urine and corresponding decreased protein content in their blood in case of diabetic individuals. Among all study groups, it was noted that 75% of control spectra show correct classification, while spectral misclassification is high amongst the subjects with Diabetes and DMCs. Interestingly, very few Diabetes and DMC plasma spectra are misclassified as control spectra. Findings demonstrate that 70% of the Diabetes subjects without complications can be correctly identified from diabetes with complications. Further, investigations could also attempt to explore the use of serum instead of plasma to reduce the spectral misclassifications as one of the abundant constituents namely clotting factors could be avoided. The outcome of RS study may be imminent for the early detection or diagnosis of DMCs.'


example = {'instruction' : 'Write a catchy title for the following paper abstract. The title should be a single sentence that accurately captures what you have done and sounds interesting to the people who work on the same or a similar topic. The title should contain the important title keywords that other researchers use when looking for literature in databases. The title should also use synonyms, broader terms, or abstractive keywords to make it more appealing and informative. Do not use words that are not related to the paper extract or the topic',
    'input' : abstract }
def formatting_func(example):
    text = f"<s>[INST]Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.{example['instruction']}\n\n### Input:{example['input']}[/INST]"
    return text
eval_prompt = formatting_func(example)
print(f'\nPrompt Template :\n\n {eval_prompt}')


Prompt Template :

 <s>[INST]Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.Write a catchy title for the following paper abstract. The title should be a single sentence that accurately captures what you have done and sounds interesting to the people who work on the same or a similar topic. The title should contain the important title keywords that other researchers use when looking for literature in databases. The title should also use synonyms, broader terms, or abstractive keywords to make it more appealing and informative. Do not use words that are not related to the paper extract or the topic

### Input:Diabetes is now one of the major public health challenges, globally. Prolonged diabetes leads to various diabetic microvascular complications (DMCs) like retinopathy, nephropathy, and neuropathy. Multiple factors are likely to be involved in predisposing diabetic individual

In [7]:
from datetime import datetime
import re

def model_seq_gen(model,temp = 0.7) : 
        pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer)
        start = datetime.now()
        sequences = pipe(
            f'{eval_prompt}' ,
            do_sample=True,
            max_new_tokens=100, 
            temperature=temp, 
            top_k=50, 
            top_p=0.95,
            num_return_sequences=1)
        extracted_title = re.sub(r'[\'"]', '', sequences[0]['generated_text'].split("[/INST]")[1])
        stop = datetime.now()
        time_taken = stop-start
        print(f"Execution Time : {time_taken}")
        return extracted_title


In [8]:
extracted_title = model_seq_gen(model_3,0.5)
display(extracted_title)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Execution Time : 0:00:05.457983


' Early Detection of Diabetic Microvascular Complications through Raman Spectroscopy: A Step Towards Improved Management and Reduced Costs'

In [9]:
extracted_title = model_seq_gen(model_3,0.6)
display(extracted_title)


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Execution Time : 0:00:03.318509


' Early Detection of Diabetic Microvascular Complications through Raman Spectroscopy: A Comparative Study'

In [10]:
extracted_title = model_seq_gen(model_3,0.7)
display(extracted_title)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Execution Time : 0:00:03.862566


' Early Diabetes Detection and Management through Raman Spectroscopy: A Comprehensive Analysis of Protein Content in Plasma and Serum Samples'

In [11]:
extracted_title = model_seq_gen(model_3,0.9)
display(extracted_title)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Execution Time : 0:00:04.274729


' Early Detection of Diabetic Microvascular Complications Using Raman Spectroscopy: A Comparative Study on Plasma Samples of Diabetes Patients and Healthy Subjects'

In [12]:
extracted_title = model_seq_gen(model_3,0.1)
display(extracted_title)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Execution Time : 0:00:03.553087


' Early Diabetes Detection through Raman Spectroscopy: A Step Towards Reducing Diabetic Microvascular Complications'

In [13]:
extracted_title = model_seq_gen(model_3,0.99)
display(extracted_title)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Execution Time : 0:00:03.411810


' The Power of Raman Spectroscopy: Accurately Detecting Diabetic Microvascular Complications through Plasma Analysis'